In [3]:
%load_ext autoreload
%autoreload 2
import adanet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
OUTPUT_DIR = './tmp/adanet'
datadir = './data'
is_tensorboard = False

In [5]:
datadir


'./data'

In [6]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump


In [7]:
cnn_layer_config={'feature_map_size':[128,]}
list_cnn_layer_filter = list(cnn_layer_config.values())[0]
list_cnn_layer_filter

[128]

In [9]:
import tensorflow as tf
import p8_util
import shutil

    
LEARNING_RATE = 0.003  #@param {type:"number"}
LEARNING_RATE = 0.03  #@param {type:"number"}

TRAIN_STEPS = 5  #@param {type:"integer"}
BATCH_SIZE = 49  #@param {type:"integer"}

ADANET_ITERATIONS = 5  #@param {type:"integer"}
NN_TYPE = 'DNN'
learn_mixture_weights=True

my_feature_columns, loss_reduction, tf_head \
= p8_util.get_tf_head("images",tuple_dimension, nClasses)

# A `Head` instance defines the loss function and metrics for `Estimators`.
# Tells Tensorfow how to compute loss function and metrics
#head = tf.contrib.estimator.multi_class_head(nClasses\
#                                             , loss_reduction=loss_reduction)
cnn_layer_config={'feature_map_size':[128]}
#cnn_layer_config=None

initialiser_name = 'xavier'
# When None then HE normal initialization takes place
initialiser_name = None
estimator = adanet.Estimator(
    head=tf_head,
    
    subnetwork_generator=p8_util.MyGenerator(
        nClasses,
        feature_columns=my_feature_columns,
        optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
        seed=p8_util.RANDOM_SEED,
        nn_type=NN_TYPE,
        dropout=0.5,
        learn_mixture_weights=False,
        layer_size=10,
        cnn_layer_config = cnn_layer_config,
        is_cnn_batch_norm = False,
        initializer=initialiser_name),
            
    adanet_lambda=.015,
    max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS,
    
    evaluator=adanet.Evaluator(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , NUM_EPOCHS
                          , tuple_dimension=tuple_dimension\
                          , batch_size=BATCH_SIZE),
        steps=None),
    config=p8_util.make_config(NN_TYPE,output_dir=OUTPUT_DIR))

*** INFO : MyGenerator : instantiation DONE!


TypeError: input_fn() missing 1 required positional argument: 'num_epochs'

In [ ]:
import time
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn= p8_util.input_fn("train", x_train, y_train\
                           , tuple_dimension=tuple_dimension\
                           , training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn= p8_util.input_fn("test", x_test, y_test\
                           , tuple_dimension=tuple_dimension\
                           , training=False, batch_size=BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=1,  
    )
)
end_time = time.time()

In [ ]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))


In [ ]:
#import tensorflow as tf
#help(tf.estimator.Estimator)

In [ ]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS

#filenale_model_check = './tmp/models/'+str(NN_TYPE)+'/model.ckpt-3'
#print_tensors_in_checkpoint_file(file_name=filenale_model_check, tensor_name='', all_tensors=False)
print("\n----------\n")
filename_model_checkpoint = './tmp/models/'+str(NN_TYPE)+'/model.ckpt-'+str(TRAIN_STEPS)
print_tensors_in_checkpoint_file(file_name=filename_model_checkpoint\
                                 , tensor_name='', all_tensors=False)


In [ ]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [ ]:
print("Architecture: {} / Accuracy= {} ".format(ensemble_architecture(results)\
                                                  , results['accuracy']))

In [ ]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNN_layer_1 | CNN_layer_2 |' : accuracy = 34% / Loss =  1.1142968

In [ ]:
#import keras
#help(keras.layers.drop)

In [ ]:
results.keys()
#is_tensorboard = False

In [ ]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [ ]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill

In [ ]:
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill